In [3]:
from platform import python_version

print(python_version())

3.10.4


In [2]:
# Imports
import os
import pandas as pd
import snscrape

In [6]:
# Setting variables to be used in format string command below
text_query = "elon musk"
since_date = "2022-04-01"
until_date = "2022-05-05"

# Using OS library to call CLI commands in Python
os.system('snscrape --jsonl --since {} twitter-search "{} until:{}"> text-query-tweets.json'.format(since_date, text_query, until_date))

2022-05-07 19:10:29.429  CRITICAL  snscrape._cli  Dumped stack and locals to /tmp/snscrape_locals_ax4tw9k6
Traceback (most recent call last):
  File "/home/bonzilla/.local/bin/snscrape", line 8, in <module>
    sys.exit(main())
  File "/home/bonzilla/.local/lib/python3.10/site-packages/snscrape/_cli.py", line 318, in main
    for i, item in enumerate(scraper.get_items(), start = 1):
  File "/home/bonzilla/.local/lib/python3.10/site-packages/snscrape/modules/twitter.py", line 1406, in get_items
    yield from self._v2_timeline_instructions_to_tweets(obj)
  File "/home/bonzilla/.local/lib/python3.10/site-packages/snscrape/modules/twitter.py", line 781, in _v2_timeline_instructions_to_tweets
    yield from self._v2_instruction_tweet_entry_to_tweet(entry['entryId'], entry['content'], obj)
  File "/home/bonzilla/.local/lib/python3.10/site-packages/snscrape/modules/twitter.py", line 804, in _v2_instruction_tweet_entry_to_tweet
    yield self._tweet_to_tweet(tweet, obj)
  File "/home/bonzilla

256

In [4]:
# Reads the json generated from the CLI command above and creates a pandas dataframe
tweets_df1 = pd.read_json('elongate0000.gz', lines=True, compression='gzip')

# Displays first 5 entries from dataframe
tweets_df1.head()

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card
0,snscrape.modules.twitter.Tweet,https://twitter.com/TanferTanferg/status/15220...,2022-05-04 23:59:59+00:00,@elonmusk @bevedoni It would make us happy if ...,@elonmusk @bevedoni It would make us happy if ...,1522003135794929665,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,1.521999e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[DogeArmy],None,None
1,snscrape.modules.twitter.Tweet,https://twitter.com/Doo0074/status/15220031346...,2022-05-04 23:59:59+00:00,@Crypto_Highland @RatsoLicksy @CliffToken @elo...,@Crypto_Highland @RatsoLicksy @CliffToken @elo...,1522003134612127746,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,2,...,NaN,None,1.522002e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None,None
2,snscrape.modules.twitter.Tweet,https://twitter.com/LarryDParkerJr/status/1522...,2022-05-04 23:59:59+00:00,@elonmusk @bevedoni Word !,@elonmusk @bevedoni Word !,1522003134557663235,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,1.521999e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None,None
3,snscrape.modules.twitter.Tweet,https://twitter.com/Coco140827/status/15220031...,2022-05-04 23:59:59+00:00,@teslaownersSV @elonmusk He was joking and her...,@teslaownersSV @elonmusk He was joking and her...,1522003134238715910,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,1.521719e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None,None
4,snscrape.modules.twitter.Tweet,https://twitter.com/tosino007/status/152200313...,2022-05-04 23:59:58+00:00,@elonmusk @bevedoni It’s very true!,@elonmusk @bevedoni It’s very true!,1522003132162658304,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,1.521999e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None,None


In [5]:
tweets_df1.shape

(25670, 29)

In [7]:
tweets_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25670 entries, 0 to 25669
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   _type             25670 non-null  object             
 1   url               25670 non-null  object             
 2   date              25670 non-null  datetime64[ns, UTC]
 3   content           25670 non-null  object             
 4   renderedContent   25670 non-null  object             
 5   id                25670 non-null  int64              
 6   user              25670 non-null  object             
 7   replyCount        25670 non-null  int64              
 8   retweetCount      25670 non-null  int64              
 9   likeCount         25670 non-null  int64              
 10  quoteCount        25670 non-null  int64              
 11  conversationId    25670 non-null  int64              
 12  lang              25670 non-null  object             
 13  s

In [8]:
# an empty df with columns
columns = ['date', 'id', 'user', 'content', 'lang', 'replyCount', 'retweetCount', 
           'likeCount', 'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers']
df = pd.DataFrame(columns=columns)
df

,date,id,content,lang,replyCount,retweetCount,likeCount,inReplyToTweetId,inReplyToUser,mentionedUsers


In [9]:
path = '/home/bonzilla/Documents/MSDS/DATA620/Final_Project'
files = sorted( os.listdir(path) )
files = [file for file in files if 'elongate' in file]
files

['elongate0000.gz',
 'elongate0001.gz',
 'elongate0002.gz',
 'elongate0003.gz',
 'elongate0004.gz',
 'elongate0005.gz',
 'elongate0006.gz',
 'elongate0007.gz',
 'elongate0008.gz',
 'elongate0009.gz',
 'elongate0010.gz',
 'elongate0011.gz',
 'elongate0012.gz',
 'elongate0013.gz',
 'elongate0014.gz',
 'elongate0015.gz',
 'elongate0016.gz',
 'elongate0017.gz',
 'elongate0018.gz']

In [11]:
num_tweets = 0
for file in files:
    dfn = pd.read_json(file, lines=True, compression='gzip')
    dfn = dfn[columns]
    num_tweets += dfn.shape[0]
    print( file + ': ' + str( dfn.shape[0] ) )
    df = pd.concat([df,dfn], ignore_index=True)
print( ' ' )
print( num_tweets )

elongate0000.gz: 25670
elongate0001.gz: 25165
elongate0002.gz: 25542
elongate0003.gz: 26000
elongate0004.gz: 26488
elongate0005.gz: 27900
elongate0006.gz: 25942
elongate0007.gz: 25982
elongate0008.gz: 26727
elongate0009.gz: 26387
elongate0010.gz: 27180
elongate0011.gz: 26539
elongate0012.gz: 26110
elongate0013.gz: 25233
elongate0014.gz: 25090
elongate0015.gz: 24734
elongate0016.gz: 25072
elongate0017.gz: 24757
elongate0018.gz: 11803
 
478321


In [12]:
df.to_csv( 'elongate_tweets.csv', index=False)